In [1]:
%load_ext autoreload
%autoreload 2

root = r"C:\Users\zwecher\Documents\mygit\new_predictor"
#predictor_path  = r"C:\Users\zwecher\Documents\mygit\Indoors_Capabilities\predictor"
#model_path = r"C:\Users\zwecher\Documents\mygit\Indoors_Capabilities\models"

from importlib import reload  # Not needed in Python 2
import logging
reload(logging)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import models
from tensorflow.keras import backend as K 

from tensorflow import keras
import sys, os
sys.path.insert(0,root)
#from predictor import predictor, config, restore
import utils
from predictor import buildings_predictor as bp
from predictor import batch_manager
from predictor import full_model_predictor as fmp
utils.set_log()
os.chdir(r"C:\Users\zwecher\Documents\mygit\new_predictor\predictor")

C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 


2019-11-22 14:48:04,980 [INFO: __init__.py 23] path: ..\logs


C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([

In [3]:
K.clear_session()
model = fmp.get_model()


2019-11-22 14:49:54,456 [INFO: full_model_predictor.py 20] 82

2019-11-22 14:49:54,652 [INFO: full_model_predictor.py 37] labels: Tensor("lab:0", shape=(None, 6560), dtype=float32)

2019-11-22 14:49:54,652 [INFO: full_model_predictor.py 38] logits: Tensor("logits/Identity:0", shape=(None, 6560), dtype=float32)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
obs (InputLayer)                [(None, 82, 80, 1)]  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 80, 78, 25)   250         obs[0][0]                        
__________________________________________________________________________________________________
max_pool_2d_0 (MaxPooling2D)    (None, 77, 75, 25)   0           conv_0[0][0] 

In [6]:

fmp.train_model(model,20)

[autoreload of tensorflow.python.keras.layers.core failed: Traceback (most recent call last):
  File "C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\importlib\__init__.py", line 147, in reload
    raise ImportError(msg.format(name), name=name)
ImportError: module utils not in sys.modules
]


Epoch 1/20
3905/3906 [============================>.] - ETA: 0s - loss: 0.2843
Epoch 00001: loss improved from inf to 0.28434, saving model to C:\Users\zwecher\Documents\mygit\new_predictor\models\models_6260\model-01-0.2843.hdf5
3906/3906 [==============================] - 2345s 600ms/step - loss: 0.2843 - val_loss: 0.2844
Epoch 2/20
3905/3906 [============================>.] - ETA: 0s - loss: 0.2802
Epoch 00002: loss improved from 0.28434 to 0.28017, saving model to C:\Users\zwecher\Documents\mygit\new_predictor\models\models_6260\model-02-0.2802.hdf5
3906/3906 [==============================] - 2335s 598ms/step - loss: 0.2802 - val_loss: 0.2806
Epoch 3/20
3905/3906 [============================>.] - ETA: 0s - loss: 0.2783
Epoch 00003: loss improved from 0.28017 to 0.27830, saving model to C:\Users\zwecher\Documents\mygit\new_predictor\models\models_6260\model-03-0.2783.hdf5
3906/3906 [==============================] - 2342s 600ms/step - loss: 0.2783 - val_loss: 0.2759
Epoch 4/20
390

In [ ]:
metadata_df = batch_manager.create_metadata_df()
bm = batch_manager.BatchManager(metadata_df,test_num=256)
x_test,y_test  =  bm.get_test_batch()


In [ ]:
%matplotlib notebook
plt.imshow(x_test[0][0].squeeze())

In [ ]:
pd.Series(y_test[0].flatten()).value_counts()

In [ ]:
#func = utils.get_inermediate_layers_func(model,["flatten"])
for i in range(20):
    x,y=  bm.get_next_train_batch(256)
   
    model.train_on_batch(x,y)
    if i%1 == 0:
        loss = model.evaluate(x_test,y_test)
        print(loss)

In [ ]:
%matplotlib notebook
func= utils.get_inermediate_layers_func(model, ["obs","logits","loss_for_logits","lab","mask","masked_loss"])
obs,logits,loss_for_logits,labels,mask,masked_loss= func((x_test,y_test))
#plt.hist(q[0].flatten(),bins=1000)


In [ ]:
pd.Series(mask.flatten()).value_counts()

In [ ]:
mask.shape

In [ ]:
%matplotlib notebook
pd.Series(labels[0]).value_counts()

In [ ]:
(y_test[0]==0).sum()

In [ ]:
model.get_losses_for([x_test])

In [ ]:
i=100
%matplotlib notebook
plt.subplot(221)
plt.imshow(mask[i].reshape(82,80))
plt.subplot(222)
plt.imshow(obs[i].reshape(82,80))
plt.subplot(223)
plt.imshow(utils.logits_to_probs(logits[i]).reshape(82,80))
plt.subplot(224)
plt.imshow(labels[i].reshape(82,80))



In [ ]:
masked_loss.sum(axis=1)/(masked_loss>0).sum(axis=1)

In [ ]:
bp.train_model(model,1)

In [ ]:
#bp.train_model(model,3)
model.save(r"C:\Users\zwecher\Documents\mygit\new_predictor\models\models_6260\model_tmp.hdf5")

In [5]:
q = tf.keras.models.load_model(r"C:\Users\zwecher\Documents\mygit\new_predictor\models\models_6260\model-01-0.3152.hdf5",compile=False)
ao = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = fmp.loss
q.compile(optimizer=ao, loss=loss)
#func = utils.get_inermediate_layers_func(q,["dense"])

TypeError: 'str' object is not callable

In [ ]:
func = utils.get_inermediate_layers_func(model,["dense"])
v=func(x)

In [ ]:
v[0].shape

In [ ]:
f=model.get_layer("bn")

In [ ]:
f.get_weights()

In [ ]:
func = utils.get_inermediate_layers_func(model,["flatten"])
v=func(x_test)

In [ ]:
%matplotlib notebook
i=80
plt.subplot(131)
plt.imshow(utils.logits_to_probs(v[0][i]).reshape(62,60))
plt.subplot(132)
plt.imshow(x_test[i].squeeze())
plt.subplot(133)
plt.imshow(y_test[i].reshape(62,60))


In [ ]:
%matplotlib notebook
plt.hist(v[0].flatten(),bins=1000)

In [ ]:
3+4

In [ ]:
q.compile()

In [ ]:
model.get_config()

In [ ]:
(v[0]<0).mean()

In [ ]:
%matplotlib tk
plt.hist(v[0][0].flatten())

In [ ]:
from datetime import datetime


In [ ]:
datetime.now()-datetime(2017,2,6)

In [ ]:
q = np.random.randn(5,6,7)
l = tf.keras.layers.Conv2D(q,kernel_size=10,kernel_initializer="ones")

In [ ]:
l.

In [ ]:
import cv2

In [ ]:
%matplotlib notebook
im = labels[0].reshape(62,60)
plt.imshow(cv2.resize(im,(21,20)))

In [ ]:
pd.Series(cv2.resize(im,(30,30)).flatten()).value_counts()